In [4]:
import numpy as np
import pandas as pd
from collections import defaultdict
from scipy.stats import hmean
from scipy.spatial.distance import cdist
from scipy import stats
import numbers

# Exploratory Data Analysis

In [31]:
df = pd.read_csv('data/Students_Performance_mv.csv')
df.head()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


In [32]:
# Drop numerical columns
df.drop(columns=['math score','reading score','writing score'], inplace=True)

In [33]:
# Find the number of missing values per column.
df.isnull().sum()

gender                          0
race/ethnicity                 11
parental level of education    21
lunch                          12
test preparation course         4
dtype: int64

In [12]:
df.columns

Index(['gender', 'race/ethnicity', 'parental level of education', 'lunch',
       'test preparation course'],
      dtype='object')

# Starting the Algorithm

In [34]:
def ExtractCompleteTuples(df):
    # getting the rows without null values
    CT = df.dropna()
    return CT   # CT.shape #(959, 5)

In [35]:
def ExtractInCompleteTuples(df):
    # getting only the rows with null values
    ICT = df[df.isnull().any(axis=1)]
    # print(ICT.shape) #(41, 5)
    return ICT.values

In [37]:
from math import log,e

# Entropy weight method (EWM)
def ComputeAttributeWeights(CT):
    n = CT.shape[0] # the number of rows in complete tuples
    s = CT.shape[1] # the number of columns

    # 1- Normalizing data(just numerocal cols)

    # 2-1 Calculating the entropy of each numerical attribute   
    
    # 2-2 Calculating the entropy of each categorical attribute 
    def entropy(labels, base=None):
        vc = pd.Series(labels).value_counts(normalize=True, sort=False)
        base = e if base is None else base
        return -(vc * np.log(vc)/np.log(base)).sum()

    E = []          # [0.6924027159890356, 1.5185039737243646, 1.71940544072419, 0.6502094546756849, 0.6508318554230292]
    for column in CT:
        E.append(entropy(CT[column], base=None))
    # 3- Determining the weight of each attribute
    w = [0] * s     # [-1.3295556932195063, 2.241176844063399, 3.1095515115596424, -1.5119314608568568, -1.5092412015466796]
    # TODO what is k?
    k = s
    sum = 0
    for i in range(k):
        sum += E[i]

    for i in range(s):
        w[i] = (1 - E[i]) / (k - sum)

    return w

In [38]:
def SortInCompleteTuples(ICT, r):
    # Convert list to npArray
    r = np.array(r)

    # Arg sort
    argSort = np.argsort(r) # it sorts r, and returns corresponding indexes
    
    # Create new empty npArray for sorted ICT
    sortedICT = np.copy(ICT)
    for index in range(argSort.size):
        sortedICT[index] = ICT[argSort[index]]
    
    return sortedICT

In [39]:
def Partition(seq, num):
    avg  = len(seq) / float(num)
    out  = []
    last = 0.0

    while last < len(seq):
        out.append(seq[int(last):int(last + avg)])
        last += avg

    return out

In [40]:
def GenerateTuplePartition(sortedICT, m):
    T = []
    T = Partition(sortedICT, m)
    return T

In [48]:
def GenerateTuplePartitions(ICT, CT, m, s):

    W = ComputeAttributeWeights(CT)
    # STEP 1
    # Calculate tuple integrity rate, according to DEFINITION 5(example)
    inCompleteRowsCount = ICT.shape[0]      # the number of ICT rows
    r = [1] * inCompleteRowsCount                      
    for i in range(inCompleteRowsCount):
        for j in range(s):
            if pd.isnull(ICT[i][j]):
                r[i] = r[i] - W[j]    
    # TODO  r (-4.350728355623041, 2.511931460856857) ?      

    # STEP 2
    # sort ICT's tuples according to their integrity rate
    sortedICT = SortInCompleteTuples(ICT, r)
    
    # STEP 3
    tuplePartitions = GenerateTuplePartition(sortedICT, m)
    return tuplePartitions # a queue of subsets

In [63]:
# Defining a function which calculates euclidean distance between two data points(numerical)
def euclideanDistance(data1, data2, length):
    distance = 0
    for x in range(length):
        distance += np.square(data1[x] - data2[x])
    return np.sqrt(distance)

In [64]:
def weighted_hamming(data):
    categories_dist = []
    
    for category in data:
        X = pd.get_dummies(data[category])
        X_mean = X * X.mean()
        X_dot = X_mean.dot(X.transpose())
        X_np = np.asarray(X_dot.replace(0,1,inplace=False))
        categories_dist.append(X_np)
        
    categories_dist = np.array(categories_dist)
    distances = hmean(categories_dist, axis=0)
    return distances

In [65]:
def distance_matrix(data, numeric_distance = "euclidean", categorical_distance = "jaccard"):
    number_of_variables = data.shape[1]
    number_of_observations = data.shape[0]

    # Get the type of each attribute (Numeric or categorical)
    is_numeric = [all(isinstance(n, numbers.Number) for n in data.iloc[:, i]) for i, x in enumerate(data)]
    is_all_categorical = sum(is_numeric) == 0

    # Replace missing values with mode for categorical columns
    for x in data:
        data[x].fillna(data[x].mode()[0], inplace=True)

    # "Dummifies" categorical variables in place
    if not (categorical_distance == 'hamming' or categorical_distance == 'weighted-hamming'):
        data = pd.get_dummies(data)
    elif categorical_distance == 'hamming':
        data = pd.DataFrame([pd.factorize(data[x])[0] for x in data]).transpose()

    if is_all_categorical:
        if categorical_distance == "weighted-hamming":
            result_matrix = weighted_hamming(data)
        else:
            result_matrix = cdist(data, data, metric=categorical_distance)

    # Fill the diagonal with NaN values
    np.fill_diagonal(result_matrix, np.nan)
    return pd.DataFrame(result_matrix)

In [66]:
def knn_impute(target, attributes, k_neighbors, aggregation_method="mode", numeric_distance="euclidean",
               categorical_distance="jaccard", missing_neighbors_threshold = 0.5):
    
    number_observations = len(target)
    is_target_numeric = all(isinstance(n, numbers.Number) for n in target)
    numberOfSamples=len(target[0])
   
    # Make sure the data are in the right format
    target = pd.DataFrame(target)
    attributes = pd.DataFrame(attributes)
    
    # Get the distance matrix and check whether no error was triggered when computing it
    distances = distance_matrix(attributes.transpose(), numeric_distance, categorical_distance)

    # Get the closest points and compute the correct aggregation method
    for j in range(numberOfSamples):
        for i, value in enumerate(target.iloc[:, j]):
            if pd.isnull(value):
                order = distances.iloc[i,:].values.argsort()[:k_neighbors]
                closest_to_target = attributes.iloc[i, order]
                target.iloc[i,j]=stats.mode(closest_to_target)[0][0]
    
    return target

In [67]:
def KNNImputation(train_set, test_set):
    knn_impute(test_set.transpose(),train_set.transpose(), k_neighbors=10)

In [68]:
def Merge(a, b):
    a=np.array(a)
    b=np.array(b)
    return np.concatenate((a, b), axis=0)

In [78]:
# Begin
CT  = ExtractCompleteTuples(df)   # this is dataframe
ICT = ExtractInCompleteTuples(df) # this is npArray

# The number of partitions
m = 5 # TODO ?
# The number of attributes
s = df.columns.size    # 5

T = GenerateTuplePartitions(ICT, CT, m, s)

# print(T[1])

CTS    = [[0]] * (m)
CTS[0] = np.array(CT.copy())

for i in range(1, m):
    KNNImputation(CTS[i-1], T[i])
    CTS[i] =  Merge(CTS[i-1], T[i])

KNNImputation(CTS[0], T[1])
CTS[1] = Merge(CTS[0], T[1])
    
print(np.array(CTS[m-1]))   



# Do cross validation
# for i in range(1, m-1):


[['female' 'group B' "bachelor's degree" 'standard' 'none']
 ['female' 'group C' 'some college' 'standard' 'completed']
 ['female' 'group B' "master's degree" 'standard' 'none']
 ...
 ['female' 'group E' "associate's degree" nan 'none']
 ['female' 'group B' "bachelor's degree" nan 'none']
 ['female' 'group E' 'some college' nan 'completed']]
